# Week 5 - Coursera Capstone Project (The Battle of Neighborhoods)

## Opening a Restaurant in Mumbai, India

 1. Build a dataframe of neighborhoods in Mumbai. 
 2. Get the Geographical coordinates of the neighborhoods.
 3. Obtain the venue data for the neighborhoods from Foursquare API.
 4. Explore the neighborhoods in Mumbai.
 5. Analyze the neighborhoods in Mumbai. 
 6. Clustering Neighborhoods.
 7. Examine and select the best cluster to open a restaurant.

In [2]:
!conda install -c conda-forge geopy --yes
get_ipython().system(u' pip install beautifulsoup4')
!pip install geocoder

Solving environment: done

# All requested packages already installed.



### 1. Build a dataframe of neighborhoods in Mumbai. 

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


#### Scraping of the data from Wikipedia page into a DataFram

In [7]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Mumbai').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

# create the list to store table data
neighborhood_list = []

#### Using of BeautifulSoup to transform the data in the table on the Wikipedia page into dataframe

In [8]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhood_list.append(row.text)

In [9]:
mu_df = pd.DataFrame({"Neighborhood": neighborhood_list})

mu_df.head()

,Neighborhood
0,List of neighbourhoods in Mumbai
1,Aarey Milk Colony
2,Agripada
3,Altamount Road
4,"Amboli, Mumbai"


In [10]:
# drop cells with a Neighborhood that is Not a Neighborhood
mu_df = mu_df[mu_df.Neighborhood != "List of neighbourhoods in Mumbai"].reset_index(drop=True)
mu_df.head()

,Neighborhood
0,Aarey Milk Colony
1,Agripada
2,Altamount Road
3,"Amboli, Mumbai"
4,Amrut Nagar


In [11]:
# print the number of rows of the  dataframe
mu_df.shape

(134, 1)

### 2.  Get the Geographical coordinates of the neighborhoods..

In [12]:
# define a function to get coordinates

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in mu_df["Neighborhood"].tolist() ]

In [14]:
coords

[[19.166115394396662, 72.85940270774424],
 [18.97628000000003, 72.82615000000004],
 [18.96433788604836, 72.80783688604835],
 [18.940170000000023, 72.83483000000007],
 [19.145160000000033, 72.84674000000007],
 [19.026140000000055, 72.86645000000004],
 [19.042830000000038, 72.92734000000007],
 [19.10301000000004, 72.89935000000008],
 [18.920251089647408, 72.82609547821569],
 [19.062940000000026, 72.92663000000005],
 [18.935410000000047, 72.83979000000005],
 [19.054370000000063, 72.84017000000006],
 [19.062080000000037, 72.86844000000008],
 [19.167850000000044, 72.83292000000006],
 [18.95130000000006, 72.82930000000005],
 [19.140292232017064, 72.804828546814],
 [18.969030000000032, 72.80384000000004],
 [18.980740000000026, 72.84072000000003],
 [19.138910000000067, 72.93817000000007],
 [18.9497029367247, 72.82732610600179],
 [18.996790000000033, 72.84654000000006],
 [19.11017000000004, 72.89171000000005],
 [18.985870000000034, 72.83108000000004],
 [18.947690000000023, 72.82697000000007],
 

In [15]:
# create temporary dataframe to populate the coordinates of the neighbourhood into Latitude and Longitude
df_mucoords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [16]:
# merge the coordinates into the bl dataframe
mu_df['Latitude'] = df_mucoords['Latitude']
mu_df['Longitude'] = df_mucoords['Longitude']

In [17]:
# print the number of rows of the  dataframe
mu_df.shape

(134, 3)

In [18]:
mu_df.head()

,Neighborhood,Latitude,Longitude
0,Aarey Milk Colony,19.166115,72.859403
1,Agripada,18.976280,72.826150
2,Altamount Road,18.964338,72.807837
3,"Amboli, Mumbai",18.940170,72.834830
4,Amrut Nagar,19.145160,72.846740


In [19]:
# save the DataFrame as CSV file
mu_df.to_csv("mu_df.csv", index=False)

### 3. Obtain the venue data for the neighborhoods from Foursquare API.

In [20]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


#### Create a map of Mumbai with neighborhoods superimposed on top.¶

In [21]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(mu_df['Latitude'], mu_df['Longitude'], mu_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

In [22]:
# save the map as HTML file
map_mumbai.save('map_mumbai.html')

### 4. Explore the neighborhoods in Mumbai.

#### Foursquare API

In [23]:
CLIENT_ID = 'XZIKFXOSMB54MQLFVPCXOZRLT1WWG0O3RN5HSPQYFIZZYSWN' # Foursquare ID
CLIENT_SECRET = 'VTSYJEZZ0MAPYSG35D0HJGZDUCAB4RHHJJQWEOZZXIYMK3BU' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XZIKFXOSMB54MQLFVPCXOZRLT1WWG0O3RN5HSPQYFIZZYSWN
CLIENT_SECRET:VTSYJEZZ0MAPYSG35D0HJGZDUCAB4RHHJJQWEOZZXIYMK3BU


#### Now, let's get the top 100 venues that are within a radius of 1000 meters.

In [29]:
# explore the top 100 venues
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
venues = []

for lat, long, neighborhood in zip(mu_df['Latitude'], mu_df['Longitude'], mu_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [30]:
# clean the json and structure it into a pandas dataframe
mu_venues = pd.DataFrame(venues)

# define the column names
mu_venues.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(mu_venues.shape)


(5646, 7)


In [41]:
mu_venues.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Aarey Milk Colony,19.166115,72.859403,The Westin Mumbai Garden City,19.172654,72.860518,Hotel
1,Aarey Milk Colony,19.166115,72.859403,The Fern - An Eco Hotel,19.162362,72.856972,Hotel
2,Aarey Milk Colony,19.166115,72.859403,PVR Cinemas,19.174016,72.860485,Multiplex
3,Aarey Milk Colony,19.166115,72.859403,Udipi Vihar,19.166498,72.854199,Vegetarian / Vegan Restaurant
4,Aarey Milk Colony,19.166115,72.859403,MadOverDonuts,19.173902,72.860185,Donut Shop


In [32]:
# how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(mu_venues.shape[0]))

5646 venues were returned by Foursquare.


#### Calculate how many venues were returned for each neighborhood?

In [33]:
mu_venues.groupby('Neighborhood').count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Aarey Milk Colony,48,48,48,48,48,48
Agripada,26,26,26,26,26,26
Altamount Road,75,75,75,75,75,75
"Amboli, Mumbai",79,79,79,79,79,79
Amrut Nagar,14,14,14,14,14,14
Antop Hill,11,11,11,11,11,11
Anushakti Nagar,6,6,6,6,6,6
Asalfa,9,9,9,9,9,9
Badhwar Park,100,100,100,100,100,100


#### How many unique categories can be curated from all the returned venues?

In [34]:
print('There are {} uniques categories.'.format(len(mu_venues['VenueCategory'].unique())))

There are 223 uniques categories.


In [42]:
# print out the list of categories
mu_venues['VenueCategory'].unique()[:50]

array(['Hotel', 'Multiplex', 'Vegetarian / Vegan Restaurant',
       'Donut Shop', 'Coffee Shop', 'Fast Food Restaurant', 'Brewery',
       'Hotel Bar', 'Breakfast Spot', 'Seafood Restaurant',
       'Shopping Mall', 'Indian Restaurant', 'Snack Place', 'Bookstore',
       'Bakery', 'Bar', 'Sandwich Place', 'Asian Restaurant', 'Farm',
       'Italian Restaurant', 'Clothing Store', 'Electronics Store',
       'Design Studio', 'Bistro', 'Food Court', 'Chinese Restaurant',
       'Middle Eastern Restaurant', 'Nightclub', 'Club House',
       'History Museum', 'Cupcake Shop', 'Movie Theater', 'Pizza Place',
       'Gym', 'Department Store', 'Zoo', 'Train Station', 'Dessert Shop',
       'Restaurant', 'Japanese Restaurant', 'Theater',
       'Salon / Barbershop', 'Other Great Outdoors',
       'Gym / Fitness Center', 'Stadium', "Men's Store", 'Café', 'Park',
       'Racetrack', 'Gastropub'], dtype=object)

In [45]:
# check if the results contain "Restaurant"
"Restaurant" in mu_venues['VenueCategory'].unique()

True

### 5. Analyze the neighborhoods in Mumbai. 

In [46]:
# one hot encoding
mu_onehot = pd.get_dummies(mu_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mu_onehot['Neighborhood'] = mu_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mu_onehot.columns[-1]] + list(mu_onehot.columns[:-1])
mu_onehot = mu_onehot[fixed_columns]

mu_onehot.head()

,Neighborhood,ATM,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Outlet Mall,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Punjabi Restaurant,Racetrack,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Store,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [47]:
# Examine the new dataframe size.
mu_onehot.shape

(5646, 224)

#### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [48]:
mu_grouped = mu_onehot.groupby('Neighborhood').mean().reset_index()
print(mu_grouped.shape)
mu_grouped

(133, 224)


,Neighborhood,ATM,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Bengali Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Business Service,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Outlet Mall,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Punjabi Restaurant,Racetrack,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Store,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.00,0.020833,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.041667,0.020833,0.000000,0.000000,0.041667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.041667,0.020833,0.000000,0.00,0.000000,0.000000,0.020833,0.000000,0.145833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

#### Now, Let's create a new dataframe for Restaurant data.

In [51]:
mu_rest = mu_grouped[["Neighborhood","Restaurant"]]
mu_rest.head()

,Neighborhood,Restaurant
0,Aarey Milk Colony,0.000000
1,Agripada,0.000000
2,Altamount Road,0.026667
3,"Amboli, Mumbai",0.000000
4,Amrut Nagar,0.000000


### 6. Clustering Neighborhoods.

#### Run k-means to cluster the neighborhood into 5 clusters.

In [52]:
# set number of clusters
kclusters = 3

mu_clustering = mu_rest.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mu_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 2], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [54]:
# add clustering labels

mu_merged = mu_rest

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mu_merged["Cluster Labels"] = kmeans.labels_

mu_merged.head() 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Restaurant,Cluster Labels
0,Aarey Milk Colony,0.000000,1
1,Agripada,0.000000,1
2,Altamount Road,0.026667,1
3,"Amboli, Mumbai",0.000000,1
4,Amrut Nagar,0.000000,1


In [55]:
# merge the bl_groupled with bl_mall to add latitude/longitude for each neighborhood
mu_merged = mu_merged.join(mu_df.set_index("Neighborhood"), on="Neighborhood")

print(mu_merged.shape)
mu_merged.head() 

(133, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.000000,1,19.166115,72.859403
1,Agripada,0.000000,1,18.976280,72.826150
2,Altamount Road,0.026667,1,18.964338,72.807837
3,"Amboli, Mumbai",0.000000,1,18.940170,72.834830
4,Amrut Nagar,0.000000,1,19.145160,72.846740


In [56]:
# sort the results by Cluster Labels
print(mu_merged.shape)
mu_merged.sort_values(["Cluster Labels"])
mu_merged

(133, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.000000,1,19.166115,72.859403
1,Agripada,0.000000,1,18.976280,72.826150
2,Altamount Road,0.026667,1,18.964338,72.807837
3,"Amboli, Mumbai",0.000000,1,18.940170,72.834830
4,Amrut Nagar,0.000000,1,19.145160,72.846740
5,Antop Hill,0.000000,1,19.026140,72.866450
6,Anushakti Nagar,0.000000,1,19.042830,72.927340
7,Asalfa,0.000000,1,19.103010,72.899350
8,Badhwar Park,0.040000,0,18.920251,72.826095
9,Baiganwadi,0.200000,2,19.062940,72.926630


#### Finally, let's visualize the resulting clusters.

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mu_merged['Latitude'], mu_merged['Longitude'], mu_merged['Neighborhood'], mu_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [58]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### Finally Let's examine Clusters to select the best cluster to open a new shopping mall.

#### Cluster 1

In [59]:
mu_merged.loc[mu_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
8,Badhwar Park,0.040000,0,18.920251,72.826095
14,Bhuleshwar,0.036364,0,18.951300,72.829300
15,Bori Bunder,0.058824,0,19.140292,72.804829
19,Cavel,0.035088,0,18.949703,72.827326
20,"Chandanwadi, Mumbai",0.050000,0,18.996790,72.846540
21,Chandivali,0.066667,0,19.110170,72.891710
22,Charni Road,0.035088,0,18.957190,72.824770
23,Chinchpokli,0.040000,0,18.985870,72.831080
24,Chira Bazaar,0.038462,0,18.947690,72.826970
27,Colaba,0.035714,0,18.915420,72.825920


#### Cluster 2

In [60]:
mu_merged.loc[mu_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.000000,1,19.166115,72.859403
1,Agripada,0.000000,1,18.976280,72.826150
2,Altamount Road,0.026667,1,18.964338,72.807837
3,"Amboli, Mumbai",0.000000,1,18.940170,72.834830
4,Amrut Nagar,0.000000,1,19.145160,72.846740
5,Antop Hill,0.000000,1,19.026140,72.866450
6,Anushakti Nagar,0.000000,1,19.042830,72.927340
7,Asalfa,0.000000,1,19.103010,72.899350
10,Ballard Estate,0.014925,1,18.935410,72.839790
11,Bandra,0.030303,1,19.054370,72.840170


#### Cluster 3

In [61]:
mu_merged.loc[mu_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
9,Baiganwadi,0.200000,2,19.062940,72.926630
109,SEEPZ,0.156250,2,19.126790,72.874040
111,Saki Naka,0.142857,2,19.126853,72.894312
123,Uttan,0.250000,2,26.866340,80.938840


### Final Analysis and Observation:

* Clusetr 1 - 2nd highest number of restaurants 
* Cluster 2 - Purple - few restaurants
* Cluster 3 - highest number of restaurants


There are few restaurants in west and south Mumbai (Cluster 2). While Cluster 3 has the highest number of restaurants, Cluster 1 has the second most number of restaurants.  Therefore, it is prudent to open new resturants in Cluster 2 neighbourhoods. Dadar and Matunga have predominantly Cluster 2 locations and may be ideal locations for new restaurants.